<a href="https://colab.research.google.com/github/dngg6688/100-Days-Of-ML-Code/blob/master/lesson5/%E3%80%8CEX5_5%EF%BC%9A%E8%87%AA%E5%BB%BACNN%E7%A5%9E%E7%B6%93%E7%B6%B2%E8%B7%AF%E8%A8%93%E7%B7%B4cifar_10_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Jul  5 02:57:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 載入必要的套件

In [3]:
import numpy as np
import torch
#建立神經網路用
import torch.nn as nn
import torch.nn.functional as F
#載入優化器
from torch import optim
#預處理資料用
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt
import matplotlib.font_manager as plt_font
import time
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from IPython import display
folder="/content/drive/MyDrive/解密AI黑盒子分享/"
twfont1 = plt_font.FontProperties(fname=folder+"字型/kaiu.ttf")

## 是否有GPU可以使用


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 定義資料預處理

In [6]:
transform=transforms.Compose([
  transforms.ToTensor(),#轉成Tensor，像素資料歸一化 像素值介於[0,1]
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))#標準化將資料轉到[-1,1]
])

## 建立DataSet類別

In [8]:
TrainDS=ImageFolder(folder+"資料集/CIFAR-10/train",transform=transform)
TrainDL=DataLoader(dataset=TrainDS,batch_size=256,shuffle=True)
TestDS=ImageFolder(folder+"資料集/CIFAR-10/test",transform=transform)
TestDL=DataLoader(dataset=TestDS,batch_size=len(TestDS),shuffle=False)

In [9]:
print('訓練集圖片資料', len(TrainDS))
print('分類名稱', TrainDS.classes)
print('分類數量', len(TrainDS.classes))#最後一層的節點數
n_class=len(TrainDS.classes)

訓練集圖片資料 50000
分類名稱 ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
分類數量 10


## 建立神經網路

In [10]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 64, 3)   #卷積層：輸入頻道數=3,卷積核數量=64,卷積核尺寸=3*3
    self.pool1 = nn.MaxPool2d(3, 3)    #池化層：池化區尺寸=3*3
    self.conv2 = nn.Conv2d(64, 32, 3)    #卷積層：輸入頻道數=64,卷積核數量=32,卷積核尺寸=3*
    self.pool2 = nn.MaxPool2d(2, 2)    #池化層：池化區尺寸=3*3
    self.fc1 = nn.Linear(32 * 4 * 4, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
    x = self.pool1(F.relu(self.conv1(x)))  #卷積後的尺寸28x28、池化後的尺吋14x14
    x = self.pool2(F.relu(self.conv2(x)))  #卷積後的尺寸10x10、池化後的尺吋5x5
    x = torch.flatten(x, 1) # flatten all dimensions except batch
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
net = Net().to(device) # inital network
optimizer =optim.Adam(net.parameters())  # create a Adam optimizer
LossFun=nn.CrossEntropyLoss()

## 開始訓練神經網路

In [11]:
net.train()
epochs = 2
step_loss=[]
starttime=time.time()
for i in range(epochs):
    for j,(data,label) in enumerate(TrainDL):
        optimizer.zero_grad()
        predicted = net(data.to(device))
        loss = LossFun(predicted,label.to(device))
        loss.backward()
        optimizer.step()
        step_loss.append(float(loss))
        print("訓練回合：",i+1,"批次：",j+1,"損失值：",loss)
        display.clear_output(wait=True)
print("總共訓練時間：",time.time()-starttime)

KeyboardInterrupt: 

## 訓練時LOSS變化

In [ ]:
plt.figure(figsize=(8,5))
plt.title("Loss隨訓練次數的變化",fontproperties=twfont1,fontsize=20)
plt.xlabel("訓練次數",fontproperties=twfont1,fontsize=20)
plt.ylabel("Loss值",fontproperties=twfont1,fontsize=20)
plt.plot(step_loss,".")
plt.show()

In [ ]:
net.eval()
correct = 0
with torch.no_grad():
    for data,label in TestDL:
        output = net(data.float().to(device))
        correct += (torch.argmax(output, dim=1) == label.to(device)).sum().item()
print("驗證資料準確度：",round(correct/len(TestDS),3))